# RAG Based on Hybrid Search (Lexical + Semantic)


## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-5-Sonnet': 'anthropic.claude-3-5-sonnet-20240620-v1:0',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Opus': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'T

## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2)

In [71]:
from langchain_aws import ChatBedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [72]:
llm_text = ChatBedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-5-Sonnet"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens": 1024,
        "stop_sequences": ["\n\nHuman"],
    },
    #streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
llm_text

ChatBedrock(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f2152099db0>], client=<botocore.client.BedrockRuntime object at 0x7f21d1c9f2b0>, model_id='anthropic.claude-3-5-sonnet-20240620-v1:0', model_kwargs={'max_tokens': 1024, 'stop_sequences': ['\n\nHuman']})

### Embedding 모델 선택

In [73]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [74]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):

    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain_aws import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Text-Embeddings-V2"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [75]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. LangChainOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [76]:
import boto3
from utils.ssm import parameter_store

In [77]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [78]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [79]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [80]:
index_name = opensearch_user_password = pm.get_params(
    key="opensearch_index_name",
    enc=True
)

print (f'index_name: {index_name}')

index_name: v01-genai-poc-parent-doc-retriever


### OpenSearch Client 생성

In [81]:
from utils.opensearch import opensearch_utils

In [82]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 4. Retriever based on Hybrid Search 정의
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- 본 샘플코드 에서는 **Hybrid Search based Retriever**를 **정의**합니다. 

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) Sematic serch를 통해 각 document별 relevant score 산출
- (2) Lexical search를 통해 각 document별 relevant score 산출
- (3-1) Rank-fusion 방식이 "simple weighted" 일 경우
    - 산출된 score에 대한 normalization 수행
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (3-2) Rank-fusion 방식이 "Reciprocal Rank Fusion (RRF)" 일 경우
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**
    - ![rrf.png](../../../10_advanced_question_answering/img/rrf.png)

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble

In [83]:
from utils.rag import OpenSearchHybridSearchRetriever

- 필터 설정 예시
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [84]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    # necessary
    os_client=os_client,
    index_name=index_name,
    llm_emb=llm_emb,
    llm_text=llm_text,
    
    # hybird-search debugger
    hybrid_search_debugger = "None", #[semantic, lexical, None]

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.51, .49], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=True,
)

### Retrieval example
- default search

In [85]:
from utils.rag import show_context_used

In [86]:
#query = "중지된 경우 이체"
query = "vidio max size?"

In [87]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.invoke(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

##############################
async_mode
##############################
True
##############################
reranker
##############################
False
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
complex_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 1.0
['. To delete a video file, click next to the name of the uploaded video file. Note The device control command must be transferred to the device to apply a video to it. &gt; Session timeout Allows the use of the session timeout feature for the Kiosk Browser. If the user does not use the device for a set time, the device deletes user information, such as the cache and cookies, in the device Kiosk Browser and goes to the main 

- update parameters

In [88]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=0,
    filter=[],
    #filter=[
    #    {"term": {"metadata.seq_num": "64"}},
    #],
    async_mode=True
)

In [89]:
#query = "중지된 경우 이체"
query = "vidio max size?"
search_hybrid_result = opensearch_hybrid_retriever.invoke(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)

##############################
async_mode
##############################
True
##############################
reranker
##############################
False
##############################
rag_fusion
##############################
False
##############################
HyDE
##############################
False
##############################
parent_document
##############################
False
##############################
complex_document
##############################
False
##############################
similar_docs_semantic
##############################

Score: 1.0
['. To delete a video file, click next to the name of the uploaded video file. Note The device control command must be transferred to the device to apply a video to it. &gt; Session timeout Allows the use of the session timeout feature for the Kiosk Browser. If the user does not use the device for a set time, the device deletes user information, such as the cache and cookies, in the device Kiosk Browser and goes to the main 

## 5. RAG

In [90]:
from textwrap import dedent

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [91]:
system_prompt = dedent(
    """
    You are a master answer bot designed to answer user's questions.
    I'm going to give you contexts which consist of texts, tables and images.
    Read the contexts carefully, because I'm going to ask you a question about it.
    """
)

human_prompt = dedent(
    """
    Here is the contexts as texts: <contexts>{contexts}</contexts>

    First, find a few paragraphs or sentences from the contexts that are most relevant to answering the question.
    Then, answer the question as much as you can.

    Skip the preamble and go straight into the answer.
    Don't insert any XML tag such as <contexts> and </contexts> when answering.
    Answer in Korean.

    Here is the question: <question>{question}</question>

    If the question cannot be answered by the contexts, say "No relevant contexts".
    """
)

### Update Search Params (Optional)

In [92]:
from utils.rag import rag_chain
from langchain.schema.output_parser import StrOutputParser

In [93]:
opensearch_hybrid_retriever.update_search_params(
    k=10,
    minimum_should_match=0,
    #filter=[
    #    {"term": {"metadata.project": "KS"}},
    #],
    filter=[],
    async_mode=True,
    verbose=False
)

### Request

In [94]:
qa = rag_chain(
    llm_text=llm_text,
    retriever=opensearch_hybrid_retriever,
    system_prompt=system_prompt,
    human_prompt=human_prompt,
    return_context=True,
    verbose=False,
    #multi_turn=True
)

In [95]:
#query = "중지된 경우 이체"
query = "vidio max size?"

response, contexts = qa.invoke(
    query = query
)

response

관련 정보를 찾았습니다:

"You can add only one video file in the MP4 or MKV format. The video file must be less than 50 MB."

답변:

비디오 파일의 최대 크기는 50MB입니다. MP4나 MKV 형식의 비디오 파일 1개만 추가할 수 있으며, 파일 크기는 50MB를 초과할 수 없습니다.

'관련 정보를 찾았습니다:\n\n"You can add only one video file in the MP4 or MKV format. The video file must be less than 50 MB."\n\n답변:\n\n비디오 파일의 최대 크기는 50MB입니다. MP4나 MKV 형식의 비디오 파일 1개만 추가할 수 있으며, 파일 크기는 50MB를 초과할 수 없습니다.'

In [70]:
show_context_used(contexts)


-----------------------------------------------
1. Chunk: 930 Characters
-----------------------------------------------
. Max TTL - The maximum amount of time, or time to live (TTL), in seconds, that a packet can exist
in a network before being discarded by the router. The default value is 64, but a number from 49 to
255 can be entered. Packet size - The size of the packet to request, in bytes. The default value is
64, but a number from 4 to 1472 can be entered. If the device user sets Packet size as 10, Interval
(ms) as 1000 ms, and Timeout (ms) as 500 ms, 10 pings are sent and the interval between each ping is
set to 1000 ms. A successful test requires all 10 pings to be received within 500 ms each. The ping
test continues even if the device user leaves the screen, but they can stop it at any time by
tapping the stop button. Otherwise, the test automatically stops once the set conditions are met. A
notification is also displayed in the notification bar for the duration of the test,

In [62]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response)

print (colored("\n\n### Contexts ###", "green"))
show_context_used(contexts)

##################################
query:  조금더 자세히 말해 볼래?
##################################


### Answer ###
해당 컨텍스트에는 조금 더 자세히 설명을 요구하는 질문에 대한 직접적인 정보가 없습니다. 하지만 컨텍스트에서 고급 프로필 기능과 모니터링에 대한 정보를 확인할 수 있습니다. 고급
프로파일 기능을 사용하려면 활성화된 Knox Suite 라이센스가 필요하고, 고급 프로파일이 할당된 디바이스의 상태를 모니터링할 수 있습니다. 또한 프로파일 유형별로 필터링하거나
디바이스를 대량으로 잠그거나 잠금 해제할 수 있습니다. 디바이스의 잠금, 등록 및 프로파일 잠금 상태를 모니터링할 수 있습니다.


### Contexts ###

-----------------------------------------------
1. Chunk: 415 Characters
-----------------------------------------------
. Clicking any link in the APP column takes you to a drill-down view for that application, letting
you see extended details like which _versions_ of the app consume the most battery power, and how
many _devices_ are running each version. Like with the expanded view, you can also use the filter
(if available) to focus on specific data, customize your table, or download the table as a CSV file
for offline viewing.
metadata:
 {'source': 'all_processed_data.json', 'seq_num': 2